In [1]:
import cpa
import scanpy as sc

[rank: 0] Global seed set to 0
2024-06-12 20:25:18,319	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-12 20:25:24,529	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_orig.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_orig.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_orig.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [4]:
adata_train.obs["split"].value_counts()

split
train    44690
test      4993
Name: count, dtype: int64

In [5]:
adata_test.obs["split"].value_counts()

split
train    4963
test      523
Name: count, dtype: int64

In [6]:
adata_ood.obs["split"].value_counts()

split
ood      8209
train     131
test       11
Name: count, dtype: int64

In [7]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test, adata_ood), label="split", keys=["train", "test", "ood"])

In [8]:
adata_combined.obs["condition_ID"].value_counts()

condition_ID
CHEMBL356066+CHEMBL2170177     3298
46245047+CHEMBL491473          3016
CHEMBL1213492+CHEMBL491473     2783
CHEMBL4297436                  2756
CHEMBL1213492+CHEMBL116438     2736
CHEMBL1213492+CHEMBL1200485    2734
CHEMBL1213492+CHEMBL460499     2692
CHEMBL1213492+CHEMBL601719     2662
CHEMBL1213492+CHEMBL1421       2421
CHEMBL1213492+CHEMBL4297436    2353
CHEMBL1421                     2343
CHEMBL1213492+CHEMBL257991     2260
CHEMBL483254+CHEMBL116438      2244
CHEMBL491473+CHEMBL2170177     2161
CHEMBL483254+CHEMBL1200485     2013
CHEMBL483254+CHEMBL4297436     1971
CHEMBL483254+CHEMBL1421        1955
CHEMBL356066+CHEMBL402548      1939
CHEMBL483254+46245047          1889
CHEMBL356066                   1869
CHEMBL483254+CHEMBL257991      1826
CHEMBL483254+CHEMBL2170177     1814
CHEMBL1213492                  1682
CHEMBL483254+CHEMBL601719      1641
CHEMBL504                      1593
CHEMBL483254                   1578
CHEMBL1213492+CHEMBL109480     1310
CHEMBL356066+CH

In [9]:
adata_combined.obs["condition_ID"] = adata_combined.obs["condition_ID"].cat.remove_unused_categories()

In [10]:
cpa.CPA.setup_anndata(
    adata_combined,
    perturbation_key="condition_ID",
    dosage_key="log_dose",
    control_group="CHEMBL504",
    batch_key=None,
    is_count_data=False,
    categorical_covariate_keys=["cell_type"],
    # deg_uns_key='rank_genes_groups_cov',
    deg_uns_cat_key="cov_drug_dose",
    max_comb_len=2,
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63520/63520 [00:00<00:00, 891115.64it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [11]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "gauss",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [12]:
model = cpa.CPA(
    adata=adata_combined,
    split_key="split",
    train_split="train",
    valid_split="valid",
    **ae_hparams,
)

[rank: 0] Global seed set to 434


In [ ]:
model.train(
    max_epochs=2000,
    use_gpu=True,
    batch_size=128,
    plan_kwargs=trainer_params,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex",
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 58.27it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB MIG 3g.20gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-5e9a7bb6-017a-5431-af98-01fe948e1fa7]


Epoch 122/2000:   6%|█▉                              | 121/2000 [1:58:55<30:44:23, 58.89s/it, v_num=1, recon=-3.36e+3, r2_mean=0.59, adv_loss=3.07, acc_pert=0.0999]

In [ ]:
model.predict(adata_combined)

In [ ]:
adata_combined.write(
    "/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_orig.h5ad"
)